In [2]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import  Flatten, Dense, Activation,Convolution2D,MaxPooling2D
from sklearn.cross_validation import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
from image_Preprocessing import load_images,resize_images
import random
%matplotlib inline

Using TensorFlow backend.
/home/edward/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
tk.Tk().withdraw()
path_Train=tkFileDialog.askdirectory()
print path_Train


/home/edward/Final Images/Training Images


In [4]:
X_Train=load_images(path_Train)

In [5]:
X_Train=np.array(X_Train)
X_Train.shape

(2000, 64, 64, 3)

In [6]:
tk.Tk().withdraw()
path_Test=tkFileDialog.askdirectory()
print path_Test


/home/edward/Final Images/Testing Images


In [7]:
X_Test=load_images(path_Test)

In [8]:
X_Test=np.array(X_Test)
X_Test.shape

(600, 64, 64, 3)

In [9]:
data_Train=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Test_v2_Part3_GroundTruth.csv")

In [10]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:600,1]
print data_Train.shape
print data_Test.shape

(2000,)
(600,)


In [11]:
y_Train = LabelEncoder().fit_transform(data_Train)
y_Train = np_utils.to_categorical(y_Train)
y_Test = LabelEncoder().fit_transform(data_Test)
y_Test = np_utils.to_categorical(y_Test)

In [12]:
print y_Train.shape
print y_Test.shape

(2000, 2)
(600, 2)


### Neural Network with two layers

In [14]:
Optimizer=Adam(lr=0.001)
objective='binary_crossentropy'
def center_normalize(x):
    return (x-K.mean(x))/K.std(x)
model=Sequential()
model.add(Activation(activation=center_normalize, input_shape=(64, 64,3)))
model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='sigmoid'))


model.add(Dense(y_Train.shape[1]))
model.add(Activation('sigmoid'))
model.compile(loss=objective,optimizer=Optimizer,metrics=['accuracy'])

In [16]:
model.fit(X_Train,y_Train,batch_size=100,nb_epoch=50,verbose=1)

/home/edward/anaconda2/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
2000/2000 [==============================] - 6s - loss: 0.5521 - acc: 0.7505     
Epoch 2/50
2000/2000 [==============================] - 0s - loss: 0.4768 - acc: 0.8130     
Epoch 3/50
2000/2000 [==============================] - 0s - loss: 0.4684 - acc: 0.8130     
Epoch 4/50
2000/2000 [==============================] - 0s - loss: 0.4652 - acc: 0.8132     
Epoch 5/50
2000/2000 [==============================] - 0s - loss: 0.4549 - acc: 0.8130     
Epoch 6/50
2000/2000 [==============================] - 0s - loss: 0.4496 - acc: 0.8162     
Epoch 7/50
2000/2000 [==============================] - 0s - loss: 0.4454 - acc: 0.8150     
Epoch 8/50
2000/2000 [==============================] - 0s - loss: 0.4434 - acc: 0.8172     
Epoch 9/50
2000/2000 [==============================] - 0s - loss: 0.4432 - acc: 0.8180     
Epoch 10/50
2000/2000 [==============================] - 0s - loss: 0.4428 - acc: 0.8167     
Epoch 11/50
2000/2000 [==============================] - 0s - loss: 0

In [17]:
scores = model.evaluate(X_Test, y_Test)
print "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)

480/600 [=======================>......] - ETA: 0s
acc: 73.08%


### Convolutional Neural Network with 4 layers

In [18]:
Optimizer=Adam(lr=0.001)
objective='binary_crossentropy'
def center_normalize(x):
    return (x-K.mean(x))/K.std(x)
model=Sequential()
#input layer
model.add(Activation(activation=center_normalize, input_shape=(64, 64,3)))
# convolutional layer
model.add(Convolution2D(32,5,5,border_mode='same',activation='relu',dim_ordering='tf'))
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Flatten())
# Relu 
model.add(Dense(128, activation='relu'))
# Sigmoid Fully connected layer
model.add(Dense(y_Train.shape[1]))
model.add(Activation('sigmoid'))
model.compile(loss=objective,optimizer=Optimizer,metrics=['accuracy'])

/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="same", activation="relu", data_format="channels_last")`
/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_last", pool_size=(2, 2))`


In [19]:
model.fit(X_Train,y_Train,batch_size=200,nb_epoch=15,verbose=1)

Epoch 1/15
2000/2000 [==============================] - 2s - loss: 1.5838 - acc: 0.7460     
Epoch 2/15
2000/2000 [==============================] - 0s - loss: 0.5977 - acc: 0.8105     
Epoch 3/15
2000/2000 [==============================] - 0s - loss: 0.5217 - acc: 0.8107     
Epoch 4/15
2000/2000 [==============================] - 0s - loss: 0.4872 - acc: 0.8122     
Epoch 5/15
2000/2000 [==============================] - 0s - loss: 0.4636 - acc: 0.8150     
Epoch 6/15
2000/2000 [==============================] - 0s - loss: 0.4470 - acc: 0.8150     
Epoch 7/15
2000/2000 [==============================] - 0s - loss: 0.4341 - acc: 0.8190     
Epoch 8/15
2000/2000 [==============================] - 0s - loss: 0.4223 - acc: 0.8210     
Epoch 9/15
2000/2000 [==============================] - 0s - loss: 0.4139 - acc: 0.8192     
Epoch 10/15
2000/2000 [==============================] - 0s - loss: 0.3973 - acc: 0.8247     
Epoch 11/15
2000/2000 [==============================] - 0s - loss: 0

In [20]:
scores = model.evaluate(X_Test, y_Test)
print "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)

600/600 [==============================] - 0s     

acc: 79.67%


### Convolutional Neural network with 6 layers

In [22]:
Optimizer=Adam(lr=0.001)
objective='binary_crossentropy'
def center_normalize(x):
    return (x-K.mean(x))/K.std(x)
model=Sequential()
#input layer
model.add(Activation(activation=center_normalize, input_shape=(64, 64,3)))
# convolutional layer
model.add(Convolution2D(32,5,5,border_mode='same',activation='relu',dim_ordering='tf'))
#pooling layer
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
# convolutional layer
model.add(Convolution2D(64,3,3,border_mode='same',activation='relu',dim_ordering='tf'))
# pooling layer
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Flatten())
# Relu 
model.add(Dense(128, activation='relu'))
# Sigmoid Fully connected layer
model.add(Dense(y_Train.shape[1]))
model.add(Activation('sigmoid'))
model.compile(loss=objective,optimizer=Optimizer,metrics=['accuracy'])

/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="same", activation="relu", data_format="channels_last")`
/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_last", pool_size=(2, 2))`
/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")`
/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_last", pool_size=(2, 2))`


In [23]:
model.fit(X_Train,y_Train,batch_size=200,nb_epoch=15,verbose=1)

Epoch 1/15
2000/2000 [==============================] - 0s - loss: 0.7390 - acc: 0.7747     
Epoch 2/15
2000/2000 [==============================] - 0s - loss: 0.5119 - acc: 0.8130     
Epoch 3/15
2000/2000 [==============================] - 0s - loss: 0.4937 - acc: 0.8130     
Epoch 4/15
2000/2000 [==============================] - 0s - loss: 0.4786 - acc: 0.8130     
Epoch 5/15
2000/2000 [==============================] - 0s - loss: 0.4632 - acc: 0.8130     
Epoch 6/15
2000/2000 [==============================] - 0s - loss: 0.4509 - acc: 0.8130     
Epoch 7/15
2000/2000 [==============================] - 0s - loss: 0.4403 - acc: 0.8130     
Epoch 8/15
2000/2000 [==============================] - 0s - loss: 0.4394 - acc: 0.8130     
Epoch 9/15
2000/2000 [==============================] - 0s - loss: 0.4492 - acc: 0.8130     
Epoch 10/15
2000/2000 [==============================] - 0s - loss: 0.4317 - acc: 0.8132     
Epoch 11/15
2000/2000 [==============================] - 0s - loss: 0

In [24]:
scores = model.evaluate(X_Test, y_Test)
print "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)

352/600 [================>.............] - ETA: 0s
acc: 80.25%
